In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np



# empty data frame with columns

In [297]:
apple_forum = pd.DataFrame({'title' : [], 'date': [], 'user':[], 'nb_views':[], 'nb_replies':[], 'link':[]})

In [ ]:
html = requests.get("https://discussions.apple.com/community/iphone/iphone_hardware").text
soup = BeautifulSoup(html, 'html.parser')
bs_object = soup.find_all('div', attrs={"class":u"discussion-description"})

## find all the elements in the page corresponding to a thread

In [298]:
for i in range(1,3):
    html = requests.get("https://discussions.apple.com/community/iphone/iphone_hardware?page="+str(i)).text
    soup = BeautifulSoup(html, 'html.parser')
    bs_object = soup.find_all('div', attrs={"class":u"discussion-description"})

    for s,index in zip(bs_object,range(len(bs_object))): 
    #links to threads
        apple_forum.loc[index+15*(i-1),'link'] = s.find('div', attrs={"class":u"discussion-title"}).find('a').get('href')
    
    #question text
        apple_forum.loc[index+15*(i-1),'title'] = s.find('div', attrs={"class":u"discussion-title"}).find('a') \
        .find('span', attrs={"class":u"discussion-description-inner-responsive u-hide--desktop"}).get_text().strip()
    
    #view number
        apple_forum.loc[index+15*(i-1),'nb_views'] = s.find('div', attrs={"class":u"discussion-metadata"}) \
        .find('div', attrs={"class":u"cell discussion-views u-hide--small"}).get_text().strip()
    
    #user name
        apple_forum.loc[index+15*(i-1),'user'] = s.find('div', attrs={"class":u"discussion-metadata"}) \
        .find('div', attrs={"class":u"cell discussion-name u-hide--small"}).get_text().strip()
    
    #latest reply and user replying
        apple_forum.loc[index+15*(i-1),'date'] = s.find('div', attrs={"class":u"discussion-metadata"}) \
        .find('div', attrs={"class":u"cell align-metadata"}).get_text().strip()
    
    #latest reply and user replying
        apple_forum.loc[index+15*(i-1),'nb_replies'] = s.find('div', attrs={"class":u"discussion-metadata"}) \
        .find_all('div', attrs={"class":"cell align-metadata"})[1].get_text().strip()

In [299]:
apple_forum

,date,link,nb_replies,nb_views,title,user
0,17 minutes ago,/thread/8243131,0 replies,3 views,Keep getting unexpected error '50' when trying...,asked by krisfromkanimbla
1,latest reply 24 minutes ago\n ...,/thread/8242901,3 replies,22 views,Is it bad to charge your phone overnight?,asked by muskaan23
2,latest reply 26 minutes ago\n ...,/thread/8243057,5 replies,19 views,why I cannot change facetime and imessage phon...,asked by negoitabuc
3,latest reply 35 minutes ago\n ...,/thread/8243005,2 replies,11 views,transfer data from iPhone7 to iPhone7,asked by FrontierFremont
4,latest reply 36 minutes ago\n ...,/thread/8242990,3 replies,9 views,Apple issue,asked by siva33
5,latest reply 38 minutes ago\n ...,/thread/8242991,3 replies,20 views,How do I know my Iphone unlocked or lock?,asked by caychau
6,54 minutes ago,/thread/8243112,0 replies,4 views,what is an ‘Unknown command’ in iMessage and h...,asked by Myrna Sanders
7,latest reply 2 hours ago\n ...,/thread/8242924,1 reply,14 views,Screen replacement and headphone mode,asked by Cheron1226
8,latest reply 2 hours ago\n ...,/thread/8218389,2 replies,73 views,If my repair and service coverage is still act...,asked by KatelynR28
9,latest reply 3 hours ago\n ...,/thread/8153750,40 replies,"5,076 views",iPhone X Lock Screen features disappear,asked by studio b


In [347]:
messages = pd.DataFrame({'thread_id' : [], 'date': [], 'message_id':[], 'message_text':[], \
                         'poster':[], 'interest':[], 'question':[], 'model':[]})

In [348]:

i=0
test = apple_forum.link
for link in test:
    html = requests.get("https://discussions.apple.com"+link).text
    soup = BeautifulSoup(html, 'html.parser')
    bs_object = soup.find('div', attrs={"class":u"j-column-wrap-l"})
    
    length = len(bs_object.find_all('div', attrs={"class":u"jive-rendered-content"}))
    
    l=0
    while l < length:
        if l == 0:
            messages.loc[i,'question'] = True
            
            ### poster name 
            messages.loc[i,'poster'] = bs_object.find('div', attrs={"class":u"j-post-author"}).find('a').text.strip()
            
            ###question text 
            #messages.loc[i,'message_text'] = bs_object.find('span',attrs = {'class':u'question-more js-toggle-title-content'}).text.strip()
            messages.loc[i,'message_text'] = bs_object.find('section',attrs = {'class':u'j-original-message'}) \
                                            .find('div',attrs = {'class':u'jive-rendered-content'}).text.strip()

            ### poster model name / OS
            try:
                messages.loc[i,'model'] = bs_object.find('p', attrs={"class":u"jive-model-info-display"}).text.strip()
            except: 
                messages.loc[i,'model'] = "not disclosed"
            ### poste date
            messages.loc[i,'date'] = bs_object.find('p', attrs={"class":u"meta-posted"}).text.strip()

            ###me too
            messages.loc[i,'interest'] = bs_object.find('a', attrs={"class":u" jive-acclaim-likelink black-button "}).text.strip()
            
            ###question id
            messages.loc[i,'message_id'] = bs_object.find('', attrs={"class":u" jive-acclaim-likelink black-button "}).get("data-object-id")
            
        else:
            messages.loc[i,'question'] = False
             
            permalink = bs_object.find_all('a', attrs={"class":u"discussionPermaLink"})
            try:
                messages.loc[i,'message_id'] = permalink[l-1].get("href")
            except:
                messages.loc[i,'message_id'] = "na"

            #user 
            users = bs_object.find_all('span', attrs={"class":u"username"})
            try:
                messages.loc[i,'poster'] = users[l-1].text.strip()
            except:
                messages.loc[i,'poster'] = "na"

            #replying to + date
            dates = bs_object.find_all('span', attrs={"class":u"j-inresponse-to"})
            try:
                messages.loc[i,'date'] = dates[l-1].text.strip()
            except: 
                messages.loc[i,'date'] = "na"
            
            #helpful answer count
            helpful = bs_object.find_all('span', attrs={"class":u"js-acclaim-container acclaim-container acclaim-like-container "})
            
            try:
                messages.loc[i,'interest'] = helpful[l-1].get("data-likes")
            except:
                messages.loc[i,'interest'] = "na"
            
            #message text
            answer = bs_object.find_all('section')
            for x in answer:
                if x.find('span', attrs={"class":u"answer-marker"}):
                    continue
                else:
                    try: 
                        messages.loc[i,"message_text"] = answer[l-1].text.strip()
                    except:
                        messages.loc[i,"message_text"] = "na"
            
        messages.loc[i,'thread_id'] = link
       #m_text = bs_object.find_all('div', attrs={"class":u"jive-rendered-content"})
       #messages.loc[i,"message_text"] = m_text[l].text.strip()
        i+=1
        l+=1
        

In [349]:
messages

,date,interest,message_id,message_text,model,poster,question,thread_id
0,"Posted on Jan 17, 2018 12:51 AM",I have this question too,32882720,Can anyone help. I have tried updating and res...,not disclosed,krisfromkanimbla,True,/thread/8243131
1,"Jan 17, 2018 1:28 AM\n\n\nin response to krisf...",0,/message/32882827#message32882827,Q: Keep getting unexpected error '50' when try...,NaN,nahomfromherzliya,False,/thread/8243131
2,"Posted on Jan 16, 2018 7:29 PM",I have this question too,32881978,"I charge my phone every night, it lasts me the...",not disclosed,muskaan23,True,/thread/8242901
3,"Jan 16, 2018 11:15 PM\n\n\nin response to musk...",0,/message/32882618#message32882618,Q: Is it bad to charge your phone overnight?\r...,NaN,Rudegar,False,/thread/8242901
4,"Jan 17, 2018 12:37 AM\n\n\nin response to musk...",0,/message/32882756#message32882756,"no, all newer batteries charge to 100% and the...",NaN,Briansyddall,False,/thread/8242901
5,"Jan 17, 2018 12:44 AM\n\n\nin response to Bria...",0,/message/32882824#message32882824,Hi No most people charge iPhone / ipad overnig...,NaN,nahomfromherzliya,False,/thread/8242901
6,"Posted on Jan 17, 2018 12:07 AM",I have this question too (1),32882547,iPhone 7 256gbLatest OS versionI already chang...,not disclosed,negoitabuc,True,/thread/8243057
7,"Jan 17, 2018 12:12 AM\n\n\nin response to nego...",0,/message/32879322#message32879322,Q: why I cannot change facetime and imessage p...,NaN,nahomfromherzliya,False,/thread/8243057
8,"Jan 17, 2018 12:26 AM\n\n\nin response to nego...",0,/message/32882750#message32882750,Peace He wants to stop sending me information....,NaN,Briansyddall,False,/thread/8243057
9,"Jan 17, 2018 12:33 AM\n\n\nin response to Bria...",0,/message/32882691#message32882691,Hi Go to settings down to second phone icon cl...,NaN,negoitabuc,False,/thread/8243057


In [264]:
bs_object.find('span', attrs={"class":u"jive-model-info-display"})

In [ ]:
html = requests.get("https://discussions.apple.com/thread/8163518").text
soup = BeautifulSoup(html, 'html.parser')
bs_object = soup.find('div', attrs={"class":u"j-column-wrap-l"})

In [ ]:
### poster name 
bs_object.find('div', attrs={"class":u"j-post-author"}).find('a').text.strip()
    
### poster model name / OS
bs_object.find('p', attrs={"class":u"jive-model-info-display"}).text.strip()

### question text
bs_object.find('span',attrs = {'class':u'question-more js-toggle-title-content'}).text.strip()
bs_object.find('section',attrs = {'class':u'j-original-message'}).find('div',attrs = {'class':u'jive-rendered-content'}).text
### poste date
bs_object.find('p', attrs={"class":u"meta-posted"}).text.strip()

###me too
bs_object.find('a', attrs={"class":u" jive-acclaim-likelink black-button "}).text.strip()

###question id
bs_object.find('', attrs={"class":u" jive-acclaim-likelink black-button "}).get("data-object-id")

### replying users
table = bs_object.find_all('span', attrs={"class":u"username"})
for x in table:
    x.text.strip()
    
#initial message + reply text: 
table = bs_object.find_all('div', attrs={"class":u"jive-rendered-content"})
for x in range(0,len(table)):
    print(x)
    print(table[x].text.strip())

#Replies only (exception for marked answers)
table = bs_object.find_all('section')
for x in table:
        if x.find('span', attrs={"class":u"answer-marker"}):
            continue
        else:
            (x)
    
    
#replying to
table = bs_object.find_all('span', attrs={"class":u"font-color-meta-light j-thread-replyto"})
for x in range(0,len(table)):
    print(x)
    print(table[x].text.strip())

##perma link to replies
table = bs_object.find_all('a', attrs={"class":u"discussionPermaLink"})
for x in range(0,len(table)):
    print(x)
    print(table[x].get("href"))

#replying to + date
table = bs_object.find_all('span', attrs={"class":u"j-inresponse-to"})
for x in range(0,len(table)):
    print(x)
    print(table[x].text.strip())
    
# helpful
table = bs_object.find_all('span', attrs={"class":u"js-acclaim-container acclaim-container acclaim-like-container "})
for x in table:
    print(x.get("data-likes"))

In [ ]:
#replying to + date
table = bs_object.find_all('span', attrs={"class":u"j-inresponse-to"})
for x in range(0,len(table)):
    print(x)
    print(table[x].text.strip())

In [333]:
bs_object.find('section',attrs = {'class':u'j-original-message'}).find('div',attrs = {'class':u'jive-rendered-content'}).text

<section class="j-original-message">
<div class="j-thread-post-wrapper">
<h1>
<span class="q-marker">Q<span class="marker-colon">:</span></span> Keep getting unexpected error '50' when trying to restore my iPhone 5c
                        </h1>
<span itemprop="text">
<!-- [DocumentBodyStart:a9bbb170-0b17-4456-8daf-dea02df610ac] --><div class="jive-rendered-content"><p>Can anyone help.</p><p style="min-height: 8pt; height: 8pt; padding: 0px;"> </p><p>I have tried updating and restoring and get the same error message.</p></div><!-- [DocumentBodyEnd:a9bbb170-0b17-4456-8daf-dea02df610ac] -->
</span>
<script>
                          var textMore = 'more';
                          var textLess = 'less';
                          var moreButton = '<button class="block-link">more</button>';
                        </script>
<!-- BEGIN PS Customization: Model Info -->
<!-- END PS Customization: Model Info -->
<p class="meta-posted">Posted on Jan 17, 2018 12:51 AM</p>
<div class="thread-acti

In [338]:
print(bs_object.prettify())

<div class="j-column-wrap-l">
 <div class="j-column j-column-l lg-margin">
  <!-- BEGIN thread messages -->
  <div class="jive-thread-messages" id="jive-thread-messages-container" itemscope="" itemtype="http://schema.org/Question" role="main">
   <!-- BEGIN thread info -->
   <div class="jive-thread-info clearfix j-rc4">
   </div>
   <!-- END thread info -->
   <!-- BEGIN original thread post / removed && (!paginator.previousPage) -->
   <div class="jive-content j-op j-rc4 " role="article">
    <div class="j-thread-post j-rc4 ">
     <!-- commenting the span as part of new design -->
     <!-- <span class="j-thread-pointer j-ui-elem"></span> -->
     <a name="8243057">
     </a>
     <a name="32882547">
     </a>
     <header class="js-original-header apple-overflow-hidden">
      <a alt="" aria-hidden="true" class="j-avatar jiveTT-hover-user u-hide--small" data-avatarid="19851917" data-externalid="" data-userid="19851917" data-username="negoitabuc" href="/people/negoitabuc" role="butt

In [ ]:
answer = bs_object.find_all('section')
            for x in answer:
                if x.find('span', attrs={"class":u"answer-marker"}):
                    continue
                else:
                    messages.loc[i,"message_text"] = answer[l-1].text.strip()